**Importing the dependencies**

In [1]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

C:\Users\ompra\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = "eastus"

In [4]:
print(resource_group)
print(workspace_name)
print(region)

ml_res
ml_diab
eastus


**Create a Resource Group from Azure Portal**

In [5]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Deploying KeyVault with name mldiabkeyvaultf9e7ffb1c0.
Deploying AppInsights with name mldiabinsights46fa0a1ecc.
Deployed AppInsights with name mldiabinsights46fa0a1ecc. Took 5.01 seconds.
Deploying StorageAccount with name mldiabstorage66a4f06d320.
Deployed KeyVault with name mldiabkeyvaultf9e7ffb1c0. Took 22.76 seconds.
Deployed StorageAccount with name mldiabstorage66a4f06d320. Took 24.14 seconds.
Deploying Workspace with name ml_diab.
Deployed Workspace with name ml_diab. Took 22.43 seconds.
Workspace ml_diab created


In [6]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [7]:
model_name='diabetes_prediction_model'

In [8]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [9]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('diab')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [10]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [11]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [12]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\ompra\AppData\Local\Temp\ipykernel_1376\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-07 21:29:15+05:30 Creating Container Registry if not exists..
2024-07-07 21:39:16+05:30 Registering the environment.
2024-07-07 21:39:19+05:30 Building image..
2024-07-07 21:46:33+05:30 Generating deployment configuration.
2024-07-07 21:46:35+05:30 Submitting deployment to compute..
2024-07-07 21:46:42+05:30 Checking the status of deployment diabetes-prediction-service..
2024-07-07 21:47:56+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
scoring_uri = service.scoring_uri

In [16]:
scoring_uri

'http://07f7b6ab-1907-4be5-a1e3-7b3b897aaa38.eastus.azurecontainer.io/score'